<a href="https://colab.research.google.com/github/SamamaSaleem/Convolutional-Neural-Network-CNN-for-Image-Classification/blob/main/Convolutional_Neural_Network_(CNN)_for_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Importing the libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

In [ ]:
print(torch.__version__)

2.4.0+cu121


## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=0, shear=0.2),
    transforms.ColorJitter(contrast=0.2, brightness=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/dataset/training_set', transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

### Preprocessing the Test set

In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/dataset/test_set', transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Part 2 - Building the CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)  # Adjust size according to the output of the flatten layer
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)  # Adjust size according to the output of the flatten layer
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

cnn = CNN()

## Part 3 - Training the CNN

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

# Training the CNN on the Training set and evaluating it on the Test set
num_epochs = 25
for epoch in range(num_epochs):
    cnn.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

### Evaluate the model on the Test set

In [ ]:
cnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = cnn(images)
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.7851


## Part 4 - Making a single prediction

In [ ]:
def predict_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path)
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    cnn.eval()
    with torch.no_grad():
        output = cnn(image)
        prediction = 'dog' if output.item() > 0.5 else 'cat'
    return prediction

In [ ]:
# Example prediction
result = predict_image('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_1.jpg')
print(result)

dog
dog
